### RLC Circuit Analysis: Complete Guide

This notebook provides a comprehensive analysis of RLC circuits including:
- Theoretical foundations
- Mathematical modeling
- Physics principles
- Circuit simulation
- Graphical analysis

## Learning Objectives:
1. Understand the behavior of R, L, and C components
2. Derive differential equations for RLC circuits
3. Analyze time and frequency domain responses
4. Simulate circuit behavior using Python
5. Visualize circuit characteristics

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy.integrate import odeint
import sympy as sp
from sympy import symbols, Function, Eq, dsolve, I, pi, exp, cos, sin
import pandas as pd

# Configure matplotlib for better plots
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")
print("Ready to explore RLC circuits!")

### Part 1: Fundamental Circuit Components

#### 1.1 Resistor (R)
- **Physical Property**: Opposes current flow, converts electrical energy to heat
- **Voltage-Current Relationship**: $V = I \times R$ (Ohm's Law)
- **Impedance**: $Z_R = R$ (frequency independent)
- **Power Dissipation**: $P = I^2R = V^2/R$

#### 1.2 Inductor (L)
- **Physical Property**: Stores energy in magnetic field, opposes current changes
- **Voltage-Current Relationship**: $V = L \times (dI/dt)$
- **Impedance**: $Z_L = j\omega L$ (frequency dependent)
- **Energy Storage**: $E = \frac{1}{2}LI^2$

#### 1.3 Capacitor (C)
- **Physical Property**: Stores energy in electric field, opposes voltage changes
- **Voltage-Current Relationship**: $I = C \times (dV/dt)$
- **Impedance**: $Z_C = \frac{1}{j\omega C}$ (frequency dependent)
- **Energy Storage**: $E = \frac{1}{2}CV^2$

In [ ]:
# Define symbolic variables for theoretical analysis
t, s, omega = symbols('t s omega', real=True, positive=True)
R, L, C = symbols('R L C', real=True, positive=True)
V0, I0 = symbols('V0 I0', real=True)

# Component impedances in frequency domain
Z_R = R
Z_L = I * omega * L # j*ω*L
Z_C = 1 / (I * omega * C) # 1/(j*ω*C)

print("Component Impedances:")
print(f"Resistor: Z_R = {Z_R}")
print(f"Inductor: Z_L = {Z_L}")
print(f"Capacitor: Z_C = {Z_C}")

# Practical component values for simulations
R_val = 10.0    # 10 Ohms
L_val = 0.01    # 10 mH
C_val = 100e-6  # 100 µF

print(f"\nPractical values for simulation:")
print(f"R = {R_val} Ω")
print(f"L = {L_val*1000} mH")
print(f"C = {C_val*1e6} µF")

### Part 2: Series RLC Circuit Analysis

#### 2.1 Circuit Configuration
In a series RLC circuit, all components share the same current $I(t)$.

#### 2.2 Kirchhoff's Voltage Law (KVL)
The sum of voltage drops equals the applied voltage:
$V_{source} = V_R + V_L + V_C$

#### 2.3 Mathematical Relationships
- $V_R = I(t) \times R$
- $V_L = L \times dI(t)/dt$
- $V_C = (\frac{1}{C}) \times \int I(t)dt$

#### 2.4 Differential Equation Derivation
Applying KVL: $V(t) = L(dI/dt) + R\times I + (\frac{1}{C})\int I dt$

Taking derivative: $dV/dt = L(\frac{d^2I}{dt^2}) + R(\frac{dI}{dt}) + \frac{I}{C}$

For constant voltage source ($dV/dt = 0$):
$L(\frac{d^2I}{dt^2}) + R(\frac{dI}{dt}) + \frac{I}{C} = 0$

In [ ]:
# Define the differential equation symbolically
i = Function('i') # Current as function of time
diff_eq = Eq(L * i(t).diff(t, 2) + R * i(t).diff(t) + i(t)/C, 0)

print("Series RLC Differential Equation:")
print(diff_eq)

# Characteristic equation: Ls² + Rs + 1/C = 0
s = symbols('s')
char_eq = L * s**2 + R * s + 1/C

print(f"\nCharacteristic equation: {char_eq} = 0")

# Solve for characteristic roots
roots = sp.solve(char_eq, s)
print(f"Characteristic roots: {roots}")

# Critical parameters
alpha = R / (2 * L) # Damping coefficient
omega_0 = 1 / sp.sqrt(L * C) # Natural frequency
omega_d = sp.sqrt(omega_0**2 - alpha**2) # Damped frequency

print(f"\nCritical Parameters:")
print(f"Damping coefficient (α): {alpha}")
print(f"Natural frequency (ω₀): {omega_0}")
print(f"Damped frequency (ωd): {omega_d}")

# Quality factor
Q = omega_0 / (2 * alpha)
print(f"Quality factor (Q): {Q}")

### Part 3: Damping Conditions

The behavior of an RLC circuit depends on the relationship between R, L, and C:

#### 3.1 Overdamped ($\omega_0^2 < \alpha^2$)
- Two distinct real roots
- No oscillation, exponential decay
- Condition: $R > 2\sqrt{L/C}$

#### 3.2 Critically Damped ($\omega_0^2 = \alpha^2$)
- One repeated real root
- Fastest approach to equilibrium without overshoot
- Condition: $R = 2\sqrt{L/C}$

#### 3.3 Underdamped ($\omega_0^2 > \alpha^2$)
- Complex conjugate roots
- Oscillatory response with exponential envelope
- Condition: $R < 2\sqrt{L/C}$

In [ ]:
# Calculate critical resistance
R_critical = 2 * sp.sqrt(L/C)
print(f"Critical resistance: R_crit = {R_critical}")

# Numerical evaluation with our component values
R_crit_val = 2 * np.sqrt(L_val/C_val)
print(f"Critical resistance (numerical): {R_crit_val:.2f} Ω")

# Classify our circuit
if R_val > R_crit_val:
    damping_type = "Overdamped"
elif R_val == R_crit_val:
    damping_type = "Critically Damped"
else:
    damping_type = "Underdamped"

print(f"Our circuit with R = {R_val} Ω is: {damping_type}")

# Calculate actual parameters
alpha_val = R_val / (2 * L_val)
omega_0_val = 1 / np.sqrt(L_val * C_val)
Q_val = omega_0_val / (2 * alpha_val)

print(f"\nNumerical Parameters:")
print(f"α = {alpha_val:.2f} rad/s")
print(f"ω₀ = {omega_0_val:.2f} rad/s")
print(f"Q = {Q_val:.2f}")

if R_val < R_crit_val:
    omega_d_val = np.sqrt(omega_0_val**2 - alpha_val**2)
    print(f"ωd = {omega_d_val:.2f} rad/s")

### Part 4: Time Domain Solutions

#### 4.1 General Solution Forms

For a series RLC circuit with initial conditions:
- Initial current: $I(0) = I_0$
- Initial capacitor voltage: $V_C(0) = V_0$

The solutions depend on the damping condition:

In [ ]:
# Define time domain solutions for each case
t_sym = symbols('t', real=True, positive=True)
A, B = symbols('A B', real=True)

# Underdamped solution (most common case)
if R_val < R_crit_val:
    print("UNDERDAMPED SOLUTION:")
    print("i(t) = e^(-αt)[A cos(ωdt) + B sin(ωdt)]")
    
    # With initial conditions I(0) = I₀, dI/dt(0) = (V₀ - I₀R)/L
    I_initial = 1.0  # Initial current (A)
    V_C_initial = 0.0  # Initial capacitor voltage (V)
    
    # Constants A and B
    A_val = I_initial
    B_val = (alpha_val * I_initial + (V_C_initial - I_initial * R_val) / L_val) / omega_d_val
    
    print(f"A = {A_val:.3f}")
    print(f"B = {B_val:.3f}")

In [ ]:
def rlc_series_response(t, R, L, C, I0=1.0, V_C0=0.0):
    """
    Calculate the current response of a series RLC circuit
    
    Parameters:
    t: time array
    R, L, C: component values
    I0: initial current
    V_C0: initial capacitor voltage
    
    Returns:
    i(t): current as function of time
    """
    
    alpha = R / (2 * L)
    omega_0 = 1 / np.sqrt(L * C)
    R_crit = 2 * np.sqrt(L / C)
    
    if R < R_crit:  # Underdamped
        omega_d = np.sqrt(omega_0**2 - alpha**2)
        A = I0
        B = (alpha * I0 + (V_C0 - I0 * R) / L) / omega_d
        current = np.exp(-alpha * t) * (A * np.cos(omega_d * t) + B * np.sin(omega_d * t))
        
    elif R == R_crit:  # Critically damped
        A = I0
        B = alpha * I0 + (V_C0 - I0 * R) / L
        current = np.exp(-alpha * t) * (A + B * t)
        
    else:  # Overdamped
        discriminant = alpha**2 - omega_0**2
        r1 = -alpha + np.sqrt(discriminant)
        r2 = -alpha - np.sqrt(discriminant)
        
        # Solve for constants using initial conditions
        A = (I0 * r2 - (V_C0 - I0 * R) / L) / (r2 - r1)
        B = I0 - A
        
        current = A * np.exp(r1 * t) + B * np.exp(r2 * t)
    
    return current

def rlc_voltages(t, current, R, L, C, V_C0=0.0):
    """
    Calculate voltages across each component
    """
    # Voltage across resistor
    V_R = current * R
    
    # Voltage across inductor
    dt = t[1] - t[0]
    dI_dt = np.gradient(current, dt)
    V_L = L * dI_dt
    
    # Voltage across capacitor
    # V_C = V_C0 + (1/C) * integral of current
    V_C = V_C0 + (1/C) * np.cumsum(current) * dt
    
    return V_R, V_L, V_C

### Part 5: Circuit Simulation and Analysis

In [ ]:
# Time array for simulation
t_sim = np.linspace(0, 0.1, 1000) # 0 to 100 ms

# Simulate current response
current = rlc_series_response(t_sim, R_val, L_val, C_val, I0=1.0, V_C0=0.0)

# Calculate component voltages
V_R, V_L, V_C = rlc_voltages(t_sim, current, R_val, L_val, C_val, V_C0=0.0)

# Create comprehensive plot
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Current vs Time
axes[0, 0].plot(t_sim * 1000, current, 'b-', linewidth=2, label='Current')
axes[0, 0].set_xlabel('Time (ms)')
axes[0, 0].set_ylabel('Current (A)')
axes[0, 0].set_title('Current Response')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Plot 2: Component Voltages
axes[0, 1].plot(t_sim * 1000, V_R, 'r-', linewidth=2, label='V_R')
axes[0, 1].plot(t_sim * 1000, V_L, 'g-', linewidth=2, label='V_L')
axes[0, 1].plot(t_sim * 1000, V_C, 'b-', linewidth=2, label='V_C')
axes[0, 1].set_xlabel('Time (ms)')
axes[0, 1].set_ylabel('Voltage (V)')
axes[0, 1].set_title('Component Voltages')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# Plot 3: Energy Analysis
E_R = V_R * current # Power in resistor (dissipated)
E_L = 0.5 * L_val * current**2 # Energy in inductor
E_C = 0.5 * C_val * V_C**2 # Energy in capacitor

axes[1, 0].plot(t_sim * 1000, E_L * 1000, 'g-', linewidth=2, label='Energy in L (mJ)')
axes[1, 0].plot(t_sim * 1000, E_C * 1000, 'b-', linewidth=2, label='Energy in C (mJ)')
axes[1, 0].plot(t_sim * 1000, (E_L + E_C) * 1000, 'k--', linewidth=2, label='Total Energy (mJ)')
axes[1, 0].set_xlabel('Time (ms)')
axes[1, 0].set_ylabel('Energy (mJ)')
axes[1, 0].set_title('Energy Storage')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Plot 4: Phase Plane (Current vs Capacitor Voltage)
axes[1, 1].plot(V_C, current, 'purple', linewidth=2)
axes[1, 1].set_xlabel('Capacitor Voltage (V)')
axes[1, 1].set_ylabel('Current (A)')
axes[1, 1].set_title('Phase Plane Plot')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Display key results
print(f"Peak current: {np.max(np.abs(current)):.3f} A")
print(f"Settlement time (5%): {t_sim[np.where(np.abs(current) < 0.05 * np.max(np.abs(current)))[0][0]] * 1000:.1f} ms")

### Part 6: Frequency Domain Analysis

#### 6.1 Transfer Function
For a series RLC circuit, the transfer function (current/voltage) is:

$H(s) = I(s)/V(s) = \frac{1}{Ls^2 + Rs + 1/C}$

#### 6.2 Frequency Response
- Magnitude: $|H(j\omega)|$
- Phase: $\angle H(j\omega)$

In [ ]:
# Define frequency range
frequencies = np.logspace(0, 4, 1000) # 1 Hz to 10 kHz
omega_freq = 2 * np.pi * frequencies

# Calculate transfer function
s_freq = 1j * omega_freq
H = 1 / (L_val * s_freq**2 + R_val * s_freq + 1/C_val)

# Magnitude and phase
magnitude_db = 20 * np.log10(np.abs(H))
phase_deg = np.angle(H) * 180 / np.pi

# Create Bode plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Magnitude plot
ax1.semilogx(frequencies, magnitude_db, 'b-', linewidth=2)
ax1.axvline(omega_0_val/(2*np.pi), color='r', linestyle='--', alpha=0.7, label=f'ω₀ = {omega_0_val/(2*np.pi):.1f} Hz')
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Magnitude (dB)')
ax1.set_title('Frequency Response - Magnitude')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Phase plot
ax2.semilogx(frequencies, phase_deg, 'r-', linewidth=2)
ax2.axvline(omega_0_val/(2*np.pi), color='r', linestyle='--', alpha=0.7, label=f'ω₀ = {omega_0_val/(2*np.pi):.1f} Hz')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Phase (degrees)')
ax2.set_title('Frequency Response - Phase')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

# Find resonant frequency and bandwidth
resonant_freq = omega_0_val / (2 * np.pi)
bandwidth = R_val / (2 * np.pi * L_val)

print(f"Resonant frequency: {resonant_freq:.2f} Hz")
print(f"Bandwidth: {bandwidth:.2f} Hz")
print(f"Quality factor: {Q_val:.2f}")

### Part 7: Parameter Sensitivity Analysis

Let's analyze how changing R, L, and C affects the circuit behavior.

In [ ]:
def analyze_parameter_effects():
    """Analyze the effect of varying circuit parameters"""
    
    t_analysis = np.linspace(0, 0.05, 500)
    
    # Vary resistance
    R_values = [5, 10, 20, 50] # Ohms
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Effect of varying R
    for R_test in R_values:
        current_R = rlc_series_response(t_analysis, R_test, L_val, C_val)
        omega_0_test = 1 / np.sqrt(L_val * C_val)
        Q_test = omega_0_test * L_val / R_test
        
        axes[0, 0].plot(t_analysis * 1000, current_R, 
                         linewidth=2, label=f'R = {R_test}Ω (Q = {Q_test:.1f})')
    
    axes[0, 0].set_xlabel('Time (ms)')
    axes[0, 0].set_ylabel('Current (A)')
    axes[0, 0].set_title('Effect of Resistance Variation')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    
    # Vary inductance
    L_values = [0.005, 0.01, 0.02, 0.04] # H
    
    for L_test in L_values:
        current_L = rlc_series_response(t_analysis, R_val, L_test, C_val)
        omega_0_test = 1 / np.sqrt(L_test * C_val)
        
        axes[0, 1].plot(t_analysis * 1000, current_L, 
                         linewidth=2, label=f'L = {L_test*1000:.0f}mH (f₀ = {omega_0_test/(2*np.pi):.1f}Hz)')
    
    axes[0, 1].set_xlabel('Time (ms)')
    axes[0, 1].set_ylabel('Current (A)')
    axes[0, 1].set_title('Effect of Inductance Variation')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].legend()
    
    # Vary capacitance
    C_values = [50e-6, 100e-6, 200e-6, 400e-6] # F
    
    for C_test in C_values:
        current_C = rlc_series_response(t_analysis, R_val, L_val, C_test)
        omega_0_test = 1 / np.sqrt(L_val * C_test)
        
        axes[1, 0].plot(t_analysis * 1000, current_C, 
                         linewidth=2, label=f'C = {C_test*1e6:.0f}µF (f₀ = {omega_0_test/(2*np.pi):.1f}Hz)')
    
    axes[1, 0].set_xlabel('Time (ms)')
    axes[1, 0].set_ylabel('Current (A)')
    axes[1, 0].set_title('Effect of Capacitance Variation')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].legend()
    
    # Damping comparison
    R_damping = [2, 10, 20, 50]
    damping_labels = []
    
    for R_damp in R_damping:
        current_damp = rlc_series_response(t_analysis, R_damp, L_val, C_val)
        R_crit = 2 * np.sqrt(L_val / C_val)
        
        if R_damp < R_crit:
            damping_type = "Underdamped"
        elif R_damp == R_crit:
            damping_type = "Critical"
        else:
            damping_type = "Overdamped"
            
        axes[1, 1].plot(t_analysis * 1000, current_damp, 
                         linewidth=2, label=f'R = {R_damp}Ω ({damping_type})')
    
    axes[1, 1].set_xlabel('Time (ms)')
    axes[1, 1].set_ylabel('Current (A)')
    axes[1, 1].set_title('Damping Comparison')
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()

# Run parameter sensitivity analysis
analyze_parameter_effects()

### Part 8: Practical Applications and Design Guidelines

#### 8.1 Common Applications:
1. **Filters**: Low-pass, high-pass, band-pass, band-stop
2. **Oscillators**: LC oscillators, crystal oscillators
3. **Resonant Circuits**: Radio tuning, impedance matching
4. **Power Systems**: Power factor correction, harmonic filtering
5. **Signal Processing**: Analog filters, equalizers

#### 8.2 Design Considerations:

In [ ]:
def design_guidelines():
    """Provide practical design guidelines for RLC circuits"""
    
    print("RLC CIRCUIT DESIGN GUIDELINES")
    print("=" * 40)
    
    print("\n1. COMPONENT SELECTION:")
    print(f"    • Choose L and C to set resonant frequency: f₀ = 1/(2π√LC)")
    print(f"    • Choose R to control damping and bandwidth")
    print(f"    • For critical damping: R = 2√(L/C)")
    print(f"    • For underdamped oscillation: R < 2√(L/C)")
    
    print(f"\n2. PERFORMANCE METRICS:")
    print(f"    • Quality Factor: Q = ω₀L/R = 1/(ω₀RC)")
    print(f"    • Higher Q → Sharper resonance, longer settling time")
    print(f"    • Lower Q → Broader resonance, faster settling time")
    
    print(f"\n3. FREQUENCY RESPONSE:")
    print(f"    • 3dB Bandwidth: BW = f₀/Q")
    print(f"    • Resonant frequency: Independent of R")
    print(f"    • Phase shift: 0° at resonance")
    
    # Design example: Audio crossover filter
    print(f"\n4. DESIGN EXAMPLE - Audio Crossover (1 kHz):")
    f_crossover = 1000 # Hz
    L_design = 0.001 # 1 mH
    C_design = 1 / ((2 * np.pi * f_crossover)**2 * L_design)
    R_design = 8 # Speaker impedance
    
    Q_design = (2 * np.pi * f_crossover * L_design) / R_design
    
    print(f"    • Target frequency: {f_crossover} Hz")
    print(f"    • Inductor: {L_design*1000:.1f} mH")
    print(f"    • Capacitor: {C_design*1e6:.1f} µF")
    print(f"    • Resistance: {R_design} Ω")
    print(f"    • Quality factor: {Q_design:.2f}")
    
    # Simulate the design
    t_design = np.linspace(0, 0.01, 1000)
    current_design = rlc_series_response(t_design, R_design, L_design, C_design)
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(t_design * 1000, current_design, 'b-', linewidth=2)
    plt.xlabel('Time (ms)')
    plt.ylabel('Current (A)')
    plt.title('Audio Crossover - Time Response')
    plt.grid(True, alpha=0.3)
    
    # Frequency response
    freq_design = np.logspace(1, 4, 1000)
    omega_design = 2 * np.pi * freq_design
    s_design = 1j * omega_design
    H_design = 1 / (L_design * s_design**2 + R_design * s_design + 1/C_design)
    
    plt.subplot(1, 2, 2)
    plt.semilogx(freq_design, 20 * np.log10(np.abs(H_design)), 'r-', linewidth=2)
    plt.axvline(f_crossover, color='k', linestyle='--', alpha=0.7, label=f'f₀ = {f_crossover} Hz')
    plt.axhline(-3, color='g', linestyle='--', alpha=0.7, label='-3 dB')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.title('Audio Crossover - Frequency Response')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

design_guidelines()

### Part 9: Summary and Key Formulas

#### 9.1 Fundamental Relationships:

In [ ]:
print("RLC CIRCUIT SUMMARY")
print("=" * 50)

print("\n📐 MATHEMATICAL FORMULAS:")
print("    Differential Equation: L(d²i/dt²) + R(di/dt) + i/C = 0")
print("    Natural Frequency: ω₀ = 1/√(LC)")
print("    Damping Coefficient: α = R/(2L)")
print("    Quality Factor: Q = ω₀/(2α) = ω₀L/R")
print("    Resonant Frequency: f₀ = 1/(2π√LC)")

print("\n🔄 DAMPING CONDITIONS:")
print("    Underdamped: R < 2√(L/C), ζ < 1")
print("    Critically Damped: R = 2√(L/C), ζ = 1")
print("    Overdamped: R > 2√(L/C), ζ > 1")

print("\n📊 TIME DOMAIN SOLUTIONS:")
print("    Underdamped: i(t) = e^(-αt)[A cos(ωdt) + B sin(ωdt)]")
print("    Critical: i(t) = e^(-αt)[A + Bt]")
print("    Overdamped: i(t) = Ae^(r₁t) + Be^(r₂t)")

print("\n🌊 FREQUENCY DOMAIN:")
print("    Transfer Function: H(s) = 1/(Ls² + Rs + 1/C)")
print("    Bandwidth: BW = R/(2πL) = f₀/Q")
print("    Impedance: Z(jω) = R + jωL + 1/(jωC)")

print("\n⚡ ENERGY RELATIONSHIPS:")
print("    Inductor Energy: E_L = ½Li²")
print("    Capacitor Energy: E_C = ½Cv²")
print("    Power Dissipation: P_R = i²R")

print("\n🎯 DESIGN GUIDELINES:")
print("    • Higher Q → Sharper resonance")
print("    • Lower R → Less damping, more oscillation")
print("    • L and C determine resonant frequency")
print("    • R determines bandwidth and damping")

In [ ]:
def create_summary_plot():
    """Create a comprehensive summary plot showing all key behaviors"""
    
    t_summary = np.linspace(0, 0.1, 1000)
    
    # Different damping scenarios
    scenarios = [
        {'R': 5, 'label': 'Underdamped (Q=3.16)', 'color': 'blue'},
        {'R': 20, 'label': 'Critically Damped (Q=1.0)', 'color': 'red'},
        {'R': 50, 'label': 'Overdamped (Q=0.4)', 'color': 'green'}
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Time domain comparison
    for scenario in scenarios:
        current = rlc_series_response(t_summary, scenario['R'], L_val, C_val)
        axes[0, 0].plot(t_summary * 1000, current, 
                         color=scenario['color'], linewidth=2, label=scenario['label'])
    
    axes[0, 0].set_xlabel('Time (ms)')
    axes[0, 0].set_ylabel('Current (A)')
    axes[0, 0].set_title('Damping Comparison - Time Domain')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    
    # Frequency domain comparison
    frequencies = np.logspace(1, 4, 1000)
    omega_freq = 2 * np.pi * frequencies
    
    for scenario in scenarios:
        s_freq = 1j * omega_freq
        H = 1 / (L_val * s_freq**2 + scenario['R'] * s_freq + 1/C_val)
        magnitude_db = 20 * np.log10(np.abs(H))
        
        axes[0, 1].semilogx(frequencies, magnitude_db, 
                             color=scenario['color'], linewidth=2, label=scenario['label'])
    
    axes[0, 1].set_xlabel('Frequency (Hz)')
    axes[0, 1].set_ylabel('Magnitude (dB)')
    axes[0, 1].set_title('Frequency Response Comparison')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].legend()
    
    # Component values effect on resonant frequency
    L_range = np.logspace(-3, -1, 50) # 1mH to 100mH
    f0_vs_L = 1 / (2 * np.pi * np.sqrt(L_range * C_val))
    
    axes[1, 0].loglog(L_range * 1000, f0_vs_L, 'purple', linewidth=2)
    axes[1, 0].axvline(L_val * 1000, color='red', linestyle='--', alpha=0.7, label='Current L')
    axes[1, 0].set_xlabel('Inductance (mH)')
    axes[1, 0].set_ylabel('Resonant Frequency (Hz)')
    axes[1, 0].set_title('Resonant Frequency vs Inductance')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].legend()
    
    # Q factor vs Resistance
    R_range = np.logspace(0, 2, 100) # 1 to 100 Ohms
    Q_vs_R = omega_0_val / (2 * R_range / (2 * L_val))
    
    axes[1, 1].semilogx(R_range, Q_vs_R, 'orange', linewidth=2)
    axes[1, 1].axvline(R_val, color='red', linestyle='--', alpha=0.7, label='Current R')
    axes[1, 1].axhline(1, color='gray', linestyle=':', alpha=0.7, label='Q = 1 (Critical)')
    axes[1, 1].set_xlabel('Resistance (Ω)')
    axes[1, 1].set_ylabel('Quality Factor (Q)')
    axes[1, 1].set_title('Quality Factor vs Resistance')
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()

create_summary_plot()

print("\n🎉 RLC CIRCUIT ANALYSIS COMPLETE!")
print("This notebook covered:")
print("✓ Theoretical foundations")
print("✓ Mathematical modeling")
print("✓ Differential equation solutions")
print("✓ Time and frequency domain analysis")
print("✓ Circuit simulation")
print("✓ Parameter sensitivity")
print("✓ Practical design guidelines")